<a href="https://colab.research.google.com/github/shaila0901/Diabetic-Retinopathy/blob/main/dr_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
from PIL import Image

In [ ]:
info=pd.read_csv("/content/a. IDRiD_Disease Grading_Training Labels.csv")
info=info.iloc[: , 0:2]
info.head()

,Image name,Retinopathy grade
0,IDRiD_001,3
1,IDRiD_002,3
2,IDRiD_003,2
3,IDRiD_004,3
4,IDRiD_005,4


In [ ]:
info['category'] = info['Retinopathy grade'].map(lambda x: 1 if x > 0 else 0)
info.head()

,Image name,Retinopathy grade,category
0,IDRiD_001,3,1
1,IDRiD_002,3,1
2,IDRiD_003,2,1
3,IDRiD_004,3,1
4,IDRiD_005,4,1


In [ ]:
testinfo=pd.read_csv("/content/b. IDRiD_Disease Grading_Testing Labels.csv")
testinfo=testinfo.iloc[: , 0:2]
testinfo.head()

,Image name,Retinopathy grade
0,IDRiD_001,4
1,IDRiD_002,4
2,IDRiD_003,4
3,IDRiD_004,4
4,IDRiD_005,4


In [ ]:
testinfo['category'] = testinfo['Retinopathy grade'].map(lambda x: 1 if x > 0 else 0)
testinfo.head()

,Image name,Retinopathy grade,category
0,IDRiD_001,4,1
1,IDRiD_002,4,1
2,IDRiD_003,4,1
3,IDRiD_004,4,1
4,IDRiD_005,4,1


In [ ]:
import shutil
shutil.copytree('/content/drive/MyDrive/a. Training Set', '/content/train')
shutil.copytree('/content/drive/MyDrive/b. Testing Set' , '/content/test')

'/content/test'

In [ ]:
impathtrain = '/content/train'
impathtest = '/content/test'

In [ ]:
images_dir = impathtrain
output_dir = '/content/traincategory'  # Directory where sorted subfolders will be created

# Iterate over the rows of the DataFrame
for index, row in info.iterrows():
    image_filename = row['Image name'] + ".jpg"
    category = row['category']

    # Create subfolder if it doesn't exist
    category_dir = os.path.join(output_dir, str(category))
    os.makedirs(category_dir, exist_ok=True)

    # Move the image file to the corresponding subfolder
    source_path = os.path.join(images_dir, image_filename)
    destination_path = os.path.join(category_dir, image_filename)
    shutil.move(source_path, destination_path)

In [ ]:
imagest_dir = impathtest
outputt_dir = '/content/testcategory'  # Directory where sorted subfolders will be created

# Iterate over the rows of the DataFrame
for index, row in testinfo.iterrows():
    image_filename = row['Image name'] + ".jpg"
    category = row['category']

    # Create subfolder if it doesn't exist
    category_dir = os.path.join(outputt_dir, str(category))
    os.makedirs(category_dir, exist_ok=True)

    # Move the image file to the corresponding subfolder
    source_path = os.path.join(imagest_dir, image_filename)
    destination_path = os.path.join(category_dir, image_filename)
    shutil.move(source_path, destination_path)

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#D:\Diabetic Retinopathy\B. Disease Grading\B. Disease Grading\1. Original Images\a. Training Set
train_dir = '/content/traincategory'
training_set = train_datagen.flow_from_directory(train_dir,
                                  target_size = (64, 64),
                                  batch_size = 32,
                                  class_mode = 'binary')

Found 413 images belonging to 2 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/testcategory',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 103 images belonging to 2 classes.


In [ ]:
# Define the CNN cnn
cnn = tf.keras.Sequential()

# Convolutional tf.keras.layers
cnn.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))

cnn.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))

cnn.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Flatten the feature maps
cnn.add(tf.keras.layers.Flatten())

# Dense tf.keras.layers
cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # Binary classification, so the output layer has 1 neuron with sigmoid activation

# Compile the cnn
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the cnn summary
cnn.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 31, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 6, 6, 128)       

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
13/13 [==============================] - 100s 8s/step - loss: 0.6367 - accuracy: 0.6368 - val_loss: 0.6230 - val_accuracy: 0.6699
Epoch 2/25
13/13 [==============================] - 93s 7s/step - loss: 0.5832 - accuracy: 0.6731 - val_loss: 0.5976 - val_accuracy: 0.6505
Epoch 3/25
13/13 [==============================] - 92s 7s/step - loss: 0.5377 - accuracy: 0.7361 - val_loss: 0.6025 - val_accuracy: 0.6408
Epoch 4/25
13/13 [==============================] - 87s 7s/step - loss: 0.5232 - accuracy: 0.7215 - val_loss: 0.5690 - val_accuracy: 0.6408
Epoch 5/25
13/13 [==============================] - 91s 7s/step - loss: 0.5072 - accuracy: 0.7288 - val_loss: 0.5734 - val_accuracy: 0.6796
Epoch 6/25
13/13 [==============================] - 93s 7s/step - loss: 0.5097 - accuracy: 0.7288 - val_loss: 0.5606 - val_accuracy: 0.6796
Epoch 7/25
13/13 [==============================] - 93s 7s/step - loss: 0.5145 - accuracy: 0.7215 - val_loss: 0.5650 - val_accuracy: 0.7087
Epoch 8/25
13/13 [=

# Making a single prediction

In [ ]:
image = Image.open('/content/testcategory/1/IDRiD_005.jpg')  # Replace with the path to your input image
image = image.resize((64, 64))  # Resize to match the input shape of the model
image = np.array(image) / 255.0  # Normalize pixel values to the range [0, 1]
image = np.expand_dims(image, axis=0)
prediction = cnn.predict(image)
if prediction < 0.5:
    print("The image is predicted as Class 1.")
else:
    print("The image is predicted as Class 0.")

1/1 [==============================] - 0s 24ms/step
The image is predicted as Class 1.


In [ ]:
image = Image.open('/content/testcategory/0/IDRiD_037.jpg')  # Replace with the path to your input image
image = image.resize((64, 64))  # Resize to match the input shape of the model
image = np.array(image) / 255.0  # Normalize pixel values to the range [0, 1]
image = np.expand_dims(image, axis=0)
prediction = cnn.predict(image)
if prediction < 0.5:
    print("The image is predicted as Class 1.")
else:
    print("The image is predicted as Class 0.")

1/1 [==============================] - 0s 39ms/step
The image is predicted as Class 0.
